### Import base library

In [1]:
import keras
import pickle
from underthesea import word_tokenize
import random

/home/pham.hoang.anh/virtualenv/deep-fashion/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/pham.hoang.anh/virtualenv/deep-fashion/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator MultiLabelBinarizer from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/pham.hoang.anh/virtualenv/deep-fashion/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/pham.hoang.anh/virtualenv/deep-fashion/lib/python

## Load data from file

In [3]:
#Load
with open ('contents.pkl', 'rb') as fp:
    contents = pickle.load(fp)
    
with open ('neg.pkl', 'rb') as fp:
    neg_data = pickle.load(fp)
    
stop_list = []
with open ('stoplist.txt', 'r') as fp:
    lines = fp.readlines()
    for line in lines:
        stop_list.append(line.replace('\n', ''))
        
random.shuffle(neg_data)

with open('tags.pkl', 'rb') as f:
    tags = pickle.load(f)

In [4]:
#Processing tag
tags = [tag.lower() for tag in tags]
tags = set(tags)
tag_space = {}
for tag in tags:
    if (" " in tag) or ("-" in tag) or ("/" in tag):
        tag_space[tag] = tag.replace(" ", "_").replace("-", "_").replace("/", "_")

In [5]:
tag_space

{'application trending 2018': 'application_trending_2018',
 'cấu hình trình duyệt ie browser': 'cấu_hình_trình_duyệt_ie_browser',
 'utf-16': 'utf_16',
 'kotlin property': 'kotlin_property',
 'solid in php': 'solid_in_php',
 'vue-resource': 'vue_resource',
 'redux-thunk': 'redux_thunk',
 'trọng âm': 'trọng_âm',
 'text summarization': 'text_summarization',
 'spam sms classifier': 'spam_sms_classifier',
 'sum type': 'sum_type',
 'signin firebase with custom token': 'signin_firebase_with_custom_token',
 'nguyen tac code css': 'nguyen_tac_code_css',
 'use case diagram': 'use_case_diagram',
 'git-branch': 'git_branch',
 'acceptance test-driven development': 'acceptance_test_driven_development',
 'github la gi': 'github_la_gi',
 'functional reactive programming': 'functional_reactive_programming',
 'download mp3 from youtube': 'download_mp3_from_youtube',
 'utf-32': 'utf_32',
 'test web': 'test_web',
 'new technology': 'new_technology',
 'syntactic analyzer': 'syntactic_analyzer',
 'orientati

In [6]:
len(contents), len(neg_data), len(stop_list)

(14157, 14000, 2242)

In [7]:
pos_train_data = contents[0:4000]
pos_test_data = contents[4000:5000]
neg_train_data = neg_data[0:4000]
neg_test_data = neg_data[4000:5000]

In [17]:
pos_test_data[6]

'## Lệnh kiểm tra performance:\n`cat /proc/cpuinfo`   -  hiển thị thông tin CPU\n\n`cat /proc/meminfo`  -   hiển thị thông tin về RAM đang sử dụng\n\n`cat /proc/version`   -  hiển thị phiên bản của kernel\n\n`cat /proc/ioports`  -   xem thông tin I/O port\n\n`cat /proc/interrupts`  -  xem thông tin interrupt\n\n`cat /proc/dma` -  xem thông tin về DMA\n\n`cat /etc/redhat-release` - hiển thị phiên bản Centos\n\n`uname -a` - hiển thị các thông tin về kernel\n\n`free -m`  -   hiển thị lượng RAM còn trống\n\n`df -h`  -  hiển thị thông tin những file hệ thống nơi mỗi file thường trú hoặc tất cả những file mặc định và lệnh này có thể xem được dung lượng ổ cứng đã sử dụng và còn trống bao nhiêu.\n\n`du -sh`   -  xem dung lượng của thư mục hiện tại\n\n`du  -ah`  -  xem chi tiết dung lượng của các thư mục con, và cả các file\n\n`du -h –max-depth=1`  -    xem dung lượng các thư mục con ở cấp 1 (ngay trong thư mục hiện tại)\n\n`df` -  kiểm tra dung lượng đĩa cứng, các phân vùng đĩa\n\n`top`   -  h

## Markdown parse

In [9]:
from bs4 import BeautifulSoup
from markdown import markdown
import re
import mistune

markdown = mistune.Markdown()

In [57]:
# Preprocessing markdown
def markdown_process(content, markdown=markdown):
    html_doc = markdown(content)
    soup = BeautifulSoup(html_doc, 'html.parser')
    
    for tag in soup.find_all(['pre']):
        tag.replace_with('')
    for tag in soup.find_all(['img']):
        tag.replace_with('')
    for tag in soup.find_all(['a']):
        tag.replace_with('')
        
    text = soup.text
    text = text.replace('\n', ' ')
    text = re.sub(r'[^\w\s]',' ',text)
    text = text.lower()
    text = text.strip()
    
    for tag in tag_space:
        text = text.replace(tag, tag_space[tag])
    
    return text

# USING Scikit-Learn Tfidf

### Create data

In [46]:
X_train = []
y_train = []
X_test = []
y_test = []

for x_train_pos in pos_train_data:
    x_train_pos = markdown_process(x_train_pos)
    X_train.append(x_train_pos)
    y_train.append(1)
for x_train_neg in neg_train_data:
    x_train_neg = markdown_process(x_train_neg)
    X_train.append(x_train_neg)
    y_train.append(0)
    
for x_test_pos in pos_test_data:
    x_test_pos = markdown_process(x_test_pos)
    X_test.append(x_test_pos)
    y_test.append(1)
for x_test_neg in neg_test_data:
    x_test_neg = markdown_process(x_test_neg)
    X_test.append(x_test_neg)
    y_test.append(0)

In [47]:
print(len(X_test))

2000


## USING ViTokenize to create dictionary

In [27]:
from pyvi import ViTokenizer

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.utils import shuffle

/home/pham.hoang.anh/virtualenv/deep-fashion/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [48]:
X_train = shuffle(X_train, random_state=42)
y_train = shuffle(y_train, random_state=42)

X_test = shuffle(X_test, random_state=100)
y_test = shuffle(y_test, random_state=100)

In [49]:
print(X_train[5008],y_train[5008])

vụ nh tân việt vks giữ nguyên quan điểm  hôm nay đại diện vksnd tp hcm đã chia bài bào chữa của 10 luật sư bào chữa cho 9 bị cáo thành hai nhóm để tranh luận lại theo đó vks vẫn bảo lưu quan điểm luận tội cũng như mức đề nghị hình phạt đối với các bị cáo  theo kiểm sát viên nguyễn văn chung quan điểm truy tố của vks là hoàn toàn đúng đắn và quan điểm mà các luật sư đưa để bảo vệ cho thân chủ mình là không có cơ sở chấp nhận từ đó ông chung chia bài bào chữa của các luật sư và bị cáo ra thành hai nhóm nhóm luật sư thứ nhất không thừa nhận kết quả điều tra tại cơ quan điều tra và yêu cầu trả hồ sơ để điều tra bổ sung nhóm thứ hai thừa nhận một phần hành vi sai trái và xin giảm nhẹ hình phạt  quan điểm của luật sư nguyễn minh tâm bào chữa cho bị cáo trần phương mai là yêu cầu trả hồ sơ để điều tra bổ sung những vấn đề như xác định rõ công ty hoàng long có thực sự bị thua lỗ hay không vợ chồng mai  vân có thực sự chiếm đoạt tiền của ngân hàng tân việt chưa và bản thân ngân hàng đã bị thiệt

In [50]:
def my_tokenizer(doc):
    doc = ViTokenizer.tokenize(doc)
    doc = doc.split(" ")
    return doc

In [51]:
vectorizer = TfidfVectorizer(tokenizer=my_tokenizer, max_features=20000)

In [52]:
x_train = vectorizer.fit_transform(X_train)

In [53]:
x_test = vectorizer.transform(X_test)

In [54]:
print(x_train.shape)

(8000, 20000)


In [55]:
vectorizer.get_feature_names()

['0',
 '00',
 '000',
 '0000',
 '00000000',
 '0001',
 '0002',
 '001',
 '002',
 '003',
 '004',
 '005',
 '007',
 '01',
 '02',
 '025',
 '03',
 '031',
 '04',
 '04726',
 '05',
 '0511',
 '06',
 '07',
 '08',
 '09',
 '0a',
 '0b',
 '0px',
 '1',
 '10',
 '100',
 '1000',
 '10000',
 '100000',
 '100g',
 '100m',
 '100ms',
 '100px',
 '100vw',
 '101',
 '1011',
 '1012',
 '1015',
 '102',
 '1020',
 '102003',
 '102005',
 '1024',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '1080',
 '109',
 '10g',
 '10gb',
 '10h',
 '10h30',
 '10k',
 '10px',
 '10s',
 '11',
 '110',
 '1100',
 '11000',
 '111',
 '1110',
 '1111',
 '11111111',
 '112',
 '112003',
 '112004',
 '112005',
 '112006',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '1192001',
 '11h',
 '11m',
 '12',
 '120',
 '1200',
 '12000',
 '120000',
 '12006',
 '121',
 '1210',
 '122',
 '122003',
 '122004',
 '123',
 '1234',
 '12345',
 '123456',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '12h',
 '13',
 '130',
 '1300',
 '131',
 '1311',
 '132',
 '133',
 '

In [58]:
filename='models/vectorizer_20k_features.pkl'
pickle.dump(vectorizer, open(filename, 'wb'))
print("SAVED SUCESS")

SAVED SUCESS


# Truncated SVD for reduce dimensons

In [59]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=128, random_state=42)

In [60]:
new_x_train = svd.fit_transform(x_train)

In [61]:
new_x_test = svd.transform(x_test)

In [62]:
print(new_x_train.shape)

(8000, 128)


In [63]:
filename='models/svd_20k_to_128.pkl'
pickle.dump(svd, open(filename, 'wb'))
print("SAVED SUCESS")

SAVED SUCESS


## Create Extra tree

In [64]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import metrics
efc = ExtraTreesClassifier(n_estimators=128)
efc.fit(new_x_train, y_train)
y_pred = efc.predict(new_x_test)
score_test = metrics.accuracy_score(y_test, y_pred)
print(score_test)

0.9965


In [73]:
my_matrix = metrics.confusion_matrix(y_test, y_pred)
print(my_matrix)

[[1000    0]
 [   7  993]]


In [65]:
filename='models/extratree_with_svd_128.pkl'
pickle.dump(efc, open(filename, 'wb'))
print("SAVED SUCESS")

SAVED SUCESS
